<a href="https://colab.research.google.com/github/mscholl96/mad-recime/blob/network_LSTM/network/LSTM/instrGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

## Basic includes

In [ ]:
!pip install ipython-autotime
%load_ext autotime

time: 1.67 ms (started: 2022-03-22 21:36:28 +00:00)


In [ ]:
!pip install word2vec
!pip install ray[tune]

     |████████████████████████████████| 42 kB 1.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for word2vec: filename=word2vec-0.11.1-py2.py3-none-any.whl size=164791 sha256=917ed518cab60707885041dd61b49d676144546134d36eae7e5d54f30e880b30
  Stored in directory: /root/.cache/pip/wheels/c9/c0/d4/29d797817e268124a32b6cf8beb8b8fe87b86f099d5a049e61
Successfully built word2vec
     |████████████████████████████████| 52.7 MB 1.1 MB/s 
     |████████████████████████████████| 175 kB 86.6 MB/s 
     |████████████████████████████████| 4.1 MB 80.8 MB/s 
     |████████████████████████████████| 125 kB 97.9 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.44.0
    Uninstalling grpcio-1.44.0:
      Successfully uninstalled grpcio-1.44.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

In [ ]:
import word2vec

import pandas as pd
import numpy as np
import re
import os

import glob

import random

time: 283 ms (started: 2022-03-22 21:36:52 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

dataPath = '/content/drive/MyDrive/TP2/Datasets/Recipe1M/'
import sys
sys.path.append(dataPath)

Mounted at /content/drive/
time: 3min 15s (started: 2022-03-22 21:36:53 +00:00)


## Import Data

In [ ]:
TIMESTAMP = '2022_03_19'

time: 945 µs (started: 2022-03-22 21:40:08 +00:00)


In [ ]:
baseFrame = pd.DataFrame()

smallSet = True

if(os.path.exists(dataPath + TIMESTAMP + '/recipes_valid_full.pkl')):
  baseFrame = pd.read_pickle(dataPath + TIMESTAMP + '/recipes_valid_full.pkl')
elif(smallSet == True):
  baseFrame = baseFrame.append(pd.read_pickle(glob.glob(dataPath + TIMESTAMP +  '/recipes_valid_*.pkl')[0]))
elif(len(glob.glob(dataPath + TIMESTAMP +  '/recipes_valid_*.pkl')) != 0):
  for file in glob.glob(dataPath + TIMESTAMP +  '/recipes_valid_*.pkl'):
    if not 'full' in file:
      baseFrame = baseFrame.append(pd.read_pickle(file))

baseFrame.head()

,title,ingredients,instructions
id,,,
000033e39b,Dilly Macaroni Salad Recipe,amount unit ingredient 0 1....,0 Cook macaroni according to package direct...
000035f7ed,Gazpacho,amount unit ingredient 0 8.0 ...,0 Add the tomatoes to a food processor with...
00003a70b1,Crunchy Onion Potato Bake,amount unit ingredient 0 2...,0 Preheat oven to 350 degrees Fah...
00004320bb,Cool 'n Easy Creamy Watermelon Pie,amount unit ingredient 0 3....,0 Dissolve Jello in boiling water. 1 ...
0000631d90,Easy Tropical Beef Skillet,amount unit ingredient 0...,"0 In a large skillet, toast the coconut ove..."


time: 26.9 s (started: 2022-03-22 21:40:08 +00:00)


In [ ]:
def getAmount(row):
  return row['amount'].tolist()
def getUnit(row):
  return row['unit'].tolist()
def getIng(row):
  return row['ingredient'].tolist()

baseFrame['amount'] = np.vectorize(getAmount, otypes=[np.ndarray])(baseFrame['ingredients'])
baseFrame['unit'] = np.vectorize(getUnit, otypes=[np.ndarray])(baseFrame['ingredients'])
baseFrame['ingredient'] = np.vectorize(getIng, otypes=[np.ndarray])(baseFrame['ingredients'])
baseFrame = baseFrame.drop(columns=['ingredients'])
baseFrame.head()

,title,instructions,amount,unit,ingredient
id,,,,,
000033e39b,Dilly Macaroni Salad Recipe,0 Cook macaroni according to package direct...,"[1.0, 1.0, 0.5, 0.5, 3.0, 0.5, 1.0, 0.75, 0.5]","[cup, cup, cup, cup, tablespoon, cup, tablespo...","[elbow macaroni, american cheese, celery, gree..."
000035f7ed,Gazpacho,0 Add the tomatoes to a food processor with...,"[8.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 3.0]","[, , , , , , , , ]","[tomatoes, kosher salt, red onion, green bell ..."
00003a70b1,Crunchy Onion Potato Bake,0 Preheat oven to 350 degrees Fah...,"[2.5, 1.5, 0.25, 1.0, 8.0, 1.0, 1.0]","[, , cup, , ounce, cup, cup]","[milk, water, butter, mashed potatoes, whole k..."
00004320bb,Cool 'n Easy Creamy Watermelon Pie,0 Dissolve Jello in boiling water. 1 ...,"[3.0, 0.25, 12.0, 2.0, 1.0]","[ounce, cup, ounce, cup, ]","[watermelon gelatin, boiling water, cool whip,..."
0000631d90,Easy Tropical Beef Skillet,"0 In a large skillet, toast the coconut ove...","[0.5, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 8.0, 16.0,...","[cup, pound-mass, tablespoon, , tablespoon, ta...","[shredded coconut, lean ground beef, fresh gar..."


time: 22.2 s (started: 2022-03-22 21:40:35 +00:00)


## Imports for Learning
https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

In [ ]:
import torch

# Model
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable 
from torchsummary import summary

# Optimizer
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split

# Tokenizer
# torch padding does only support constant padding (ConstantPad1d) for 1D or non-constant padding for >1D (nn.function.pad)
from tensorflow.keras.preprocessing.sequence import pad_sequences
# keras tokenizer more powerful than torch
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from torchtext.data import get_tokenizer # https://pytorch.org/text/stable/data_utils.html

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

# hyperparameter tuning
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

time: 6.99 s (started: 2022-03-22 21:40:57 +00:00)


# Seed

In [ ]:
torch.manual_seed(1)

time: 3.41 ms (started: 2022-03-22 21:41:04 +00:00)


# Setup
https://closeheat.com/blog/pytorch-lstm-text-generation-tutorial


## Tokenization
to be checked: necessity of punctuation (maybe reintroduce later: https://stackoverflow.com/questions/49073673/include-punctuation-in-keras-tokenizer)

### Get Corpus

In [ ]:
w2v_model = word2vec.load(dataPath + 'vocab.bin')
ingredientDict = {}
for voc in w2v_model.vocab:
     # Offset by 1 so empty fields can be 0
     ingredientDict.setdefault(voc, len(ingredientDict)+1)

if 'dilly' in ingredientDict:
  print("Word exists")
else:
  print('vocab.bin not to be used as dict misses words') 

vocab.bin not to be used as dict misses words
time: 1.43 s (started: 2022-03-22 21:41:04 +00:00)


In [ ]:
def getCorpus(title, ingredient, instructions):
  titleTok = text_to_word_sequence(title)
  ingTok = text_to_word_sequence(' '.join(ingredient))
  instTok = text_to_word_sequence(' '.join(instructions))
  return np.array(ingTok + titleTok + instTok)

corpus = np.vectorize(getCorpus, otypes=[np.ndarray])(baseFrame['title'], baseFrame['ingredient'], baseFrame['instructions'])
corpus = np.concatenate(corpus)

time: 6.84 s (started: 2022-03-22 21:41:05 +00:00)


###  Sequences

In [ ]:
test = baseFrame.head(20).copy()

def getTitleSequence(title, ingredient):
  titleTok = text_to_word_sequence(title)
  ingTok = text_to_word_sequence(' '.join(ingredient))
  return {'ings': ingTok, 'title': titleTok}

titleSeq = np.vectorize(getTitleSequence, otypes=[np.ndarray])(test['title'], test['ingredient'])
titleSeq[0]

{'ings': ['elbow',
  'macaroni',
  'american',
  'cheese',
  'celery',
  'green',
  'peppers',
  'pimentos',
  'mayonnaise',
  'vinegar',
  'salt',
  'dry',
  'dill',
  'weed'],
 'title': ['dilly', 'macaroni', 'salad', 'recipe']}

time: 6.75 ms (started: 2022-03-22 21:41:12 +00:00)


In [ ]:
def getNGramSeq(seq):
  # input needs to be pre padded
  idxShift = len(seq['title'])
  ingLen = len(seq['ings'])
  fullSeq = np.array(seq['ings'] + seq['title'])
  retSeq = np.empty((0,ingLen + 1))
  for i_shift in range(idxShift):
    retSeq = np.vstack([retSeq, np.array(fullSeq[i_shift:ingLen+i_shift+1])])
  return retSeq

ngramSeq = pd.Series(np.vectorize(getNGramSeq, otypes=[np.ndarray])(titleSeq)).explode().to_numpy()
ngramSeq[0][:-1], ngramSeq[0][1:]

(array(['elbow', 'macaroni', 'american', 'cheese', 'celery', 'green',
        'peppers', 'pimentos', 'mayonnaise', 'vinegar', 'salt', 'dry',
        'dill', 'weed'], dtype='<U32'),
 array(['macaroni', 'american', 'cheese', 'celery', 'green', 'peppers',
        'pimentos', 'mayonnaise', 'vinegar', 'salt', 'dry', 'dill', 'weed',
        'dilly'], dtype='<U32'))

time: 9.57 ms (started: 2022-03-22 21:41:12 +00:00)


### Hyperparams

In [ ]:
class HyperParams():
  def __init__(self, epochs=10, batchSize=10, lr=1e-3, ratio=[0.7, 0.2, 0.1]):
    self.epochs = epochs
    self.batchSize = batchSize
    self.lr = lr
    self.ratio = ratio

    self.hidden_dim = 256 #number of features in hidden state
    self.num_layers = 1 #number of stacked lstm layers
    self.embedding_dim = 200 # embedding dimension

  def __str__(self):
    return('epochs ' + str(self.epochs) + '\n' +
    'batchSize ' + str(self.batchSize) + '\n' +
    'lr ' + str(self.lr) + '\n' +
    'ratio train|val|test' + str(self.ratio) + '\n' +
    # 'input_size ' + str(self.input_size) + '\n' +
    'hidden_dim ' + str(self.hidden_dim) + '\n' +
    'num_layers ' + str(self.num_layers) + '\n' +
    # 'num_classes ' + str(self.num_classes) + '\n' +
    'embedding_dim ' + str(self.embedding_dim) + '\n')
    

time: 6.61 ms (started: 2022-03-22 21:41:12 +00:00)


### Dataset

In [ ]:
class InstructionSet(Dataset):
    def __init__(self, hyperparams, data):
      self.hyperparams = hyperparams

      to_exclude = '\n'
      self.tokenizer = Tokenizer(filters=to_exclude)

      # dataset split into word sequences required for training
      self.wordSeq = np.vectorize(self.getSequence, otypes=[np.ndarray])(data['ingredient'], data['title'],  data['instructions'])

      # training requires same length sequences -->  padding
      self.maxSequenceLength = max([len(seq['ingTitle']) for seq in self.wordSeq])

      # list of all words in dataset
      self.words = np.concatenate(np.vectorize(self.getCorpus, otypes=[np.ndarray])(data['ingredient'], data['title'], data['instructions']))

      # tokenization corpus
      self.tokenizer.fit_on_texts(self.words)

      # indexed wordSequences (could be calculated in getter but very slow, preprocessing better)
      self.idxWords = np.vectorize(self.getIndexedSeqs, otypes=[np.ndarray])(self.wordSeq)

      # n gram sequences
      self.movWindSeq = pd.Series(np.vectorize(self.getMovWindSeq, otypes=[np.ndarray])(self.idxWords)).explode()
      self.movWindSeq.dropna(inplace=True)
      self.movWindSeq = self.movWindSeq.to_numpy()


    def getCorpus(self, ingredient, title, instructions):
      ingTok = text_to_word_sequence(' '.join(ingredient))
      titleTok = text_to_word_sequence(title)
      instTok = text_to_word_sequence(' \n '.join(instructions))
      return np.array(ingTok + titleTok + instTok)

    def getSequence(self, ingredient, title, instructions):
      ingTok = text_to_word_sequence(' '.join(ingredient))
      titleTok = text_to_word_sequence(title)
      instTok = text_to_word_sequence(' \n '.join(instructions))
      return {'ingTitle': ingTok + titleTok, 'instructions': instTok}

    def getIndexedSeqs(self, seq):
      ingTok = self.tokenizer.texts_to_sequences([seq['ingTitle']])[0]
      ingTok = pad_sequences([ingTok], maxlen=self.maxSequenceLength, padding='pre')[0] # https://arxiv.org/abs/1903.07288
      instTok = self.tokenizer.texts_to_sequences([seq['instructions']])[0]

      return {'ingTitle': ingTok, 'instructions': instTok}

    def getNGramSeq(self, seq):
      # input needs to be pre padded
      idxShift = len(seq['instructions'])
      ingLen = len(seq['ingTitle'])

      fullSeq = np.append(seq['ingTitle'], seq['instructions'])
      retSeq = np.empty((0,ingLen + 1), dtype=np.int32)

      for i_shift in range(idxShift):
        retSeq = np.vstack([retSeq, np.array(fullSeq[i_shift:ingLen+i_shift+1])])
      return retSeq

    def __len__(self):
        return len(self.idxWords)

    def __getitem__(self, index):
        return (
            torch.tensor(self.ngramSeq[index][:-1]),
            torch.tensor(self.ngramSeq[index][1:])
        )

time: 51.3 ms (started: 2022-03-22 21:41:12 +00:00)


## Model
LSTM Net: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

Embedding Net: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html

Init state: https://stats.stackexchange.com/questions/224737/best-way-to-initialize-lstm-state

<!-- ### base: https://github.com/yuchenlin/lstm_sentence_classifier/blob/master/LSTM_sentence_classifier.py -->

### base: https://www.analyticsvidhya.com/blog/2020/08/build-a-natural-language-generation-nlg-system-using-pytorch/

### base: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#lstms-in-pytorch

In [ ]:
class Model2(nn.Module):

    def __init__(self, hyperParams, dataset, device):
        super(Model3, self).__init__()

        # initialize vital params
        self.vocab_size = len(dataset.tokenizer.word_index)
        self.batchSize = hyperParams.batchSize
        self.hidden_size = hyperParams.hidden_dim
        self.device = device
        self.num_layers = hyperParams.num_layers
        
        self.word_embeddings = nn.Embedding(self.vocab_size, hyperParams.embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=hyperParams.embedding_dim, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True)
        self.linear = nn.Linear(self.hidden_size, self.vocab_size)

    def forward(self, x, hidden, cell):
        embeds = self.word_embeddings(x)

        lstm_out, (hidden, cell) = self.lstm(embeds, (hidden.detach(), cell.detach()))

        out = self.linear(lstm_out.reshape(-1, self.hidden_size))

        return out, (hidden, cell)

    # def init_hidden(self, batchSize=None):
    #     ''' initializes hidden state '''
    #     # Create two new tensors with sizes num_layers x batchSize x hidden_dim,
    #     # initialized to zero, for hidden state and cell state of LSTM
    #     weight = next(self.parameters()).data

    #     batchSize = self.batchSize if batchSize == None else batchSize

    #     hidden = (weight.new(self.num_layers, batchSize, self.hidden_dim).zero_().to(self.device),
    #               weight.new(self.num_layers, batchSize, self.hidden_dim).zero_().to(self.device))
        
    #     return hidden

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return hidden, cell

time: 20.9 ms (started: 2022-03-22 21:41:12 +00:00)


In [ ]:
class Model3(nn.Module):

    def __init__(self, hyperParams, dataset, device):
        super(Model3, self).__init__()

        # initialize vital params
        self.vocab_size = len(dataset.tokenizer.word_index)
        self.batchSize = hyperParams.batchSize
        self.hidden_size = hyperParams.hidden_dim
        self.device = device
        self.num_layers = hyperParams.num_layers
        
        self.word_embeddings = nn.Embedding(self.vocab_size, hyperParams.embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=hyperParams.embedding_dim, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True)
        self.linear = nn.Linear(self.hidden_size, self.vocab_size)

    def forward(self, x, hidden, cell):
        embeds = self.word_embeddings(x)

        lstm_out, (hidden, cell) = self.lstm(embeds, (hidden.detach(), cell.detach()))

        out = self.linear(lstm_out.reshape(-1, self.hidden_size))

        return out, (hidden, cell)

    # def init_hidden(self, batchSize=None):
    #     ''' initializes hidden state '''
    #     # Create two new tensors with sizes num_layers x batchSize x hidden_dim,
    #     # initialized to zero, for hidden state and cell state of LSTM
    #     weight = next(self.parameters()).data

    #     batchSize = self.batchSize if batchSize == None else batchSize

    #     hidden = (weight.new(self.num_layers, batchSize, self.hidden_dim).zero_().to(self.device),
    #               weight.new(self.num_layers, batchSize, self.hidden_dim).zero_().to(self.device))
        
    #     return hidden

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return hidden, cell

time: 20.7 ms (started: 2022-03-22 21:41:12 +00:00)


## Training
mixture of 
* https://pytorch.org/tutorials/beginner/introyt/trainingyt.html
* https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html
* https://stackoverflow.com/questions/67295494/correct-validation-loss-in-pytorch


In [ ]:
def train_epoch(epoch, batchSize, model, criterion, optimizer, train_loader, device, writer):
  running_loss = 0.
  correct = 0
  total = 0

  h,c = model.init_hidden(batchSize)

  model.train()

  for batch, (inputs, labels) in enumerate(train_loader):
    if epoch == 0 and batch == 0:
      writer.add_graph(model, input_to_model=(inputs.to(device), h, c), verbose=False)

    # assign inputs and labels to device
    inputs, labels = inputs.to(device), labels.to(device)

    # detach hidden states
    # h = tuple([each.data for each in h])

    # clear gradients
    optimizer.zero_grad()

    # batch prediction
    outputs, (h,c) = model(inputs, h, c)
    labels = labels.long()

    # loss computation
    loss = criterion(outputs, labels.view(-1))

    # calc backward gradients
    loss.backward()

    # run optimizer
    optimizer.step()

    # print statistics
    running_loss += loss.item()

    # _, predicted = outputs.max(1)
    # print(outputs.shape)
    # print(predicted.shape)
    # total += labels.size(0)
    # correct += predicted.eq(labels).sum().item()

  print("Epoch: %d, loss: %1.5f" % (epoch+1, running_loss / len(train_loader)))
  return( running_loss / len(train_loader))


time: 18.6 ms (started: 2022-03-22 21:41:12 +00:00)


In [ ]:
def val_epoch(epoch, batchSize, model, criterion, optimizer, val_loader, device, writer):
  # Validation Loss
  correct = 0                                               
  total = 0                                                 
  running_loss = 0.0    

  h,c = model.init_hidden(batchSize)
      
  model.eval() # what does it do
  with torch.no_grad(): # what does it do
    for batch, (inputs, labels) in enumerate(val_loader):
      # assign inputs and labels to device
      inputs, labels = inputs.to(device), labels.to(device)

      # batch prediction (alternative: forward)
      outputs, (h,c) = model(inputs, h, c)
      labels = labels.long()

      # loss computation
      loss = criterion(outputs, labels.view(-1))

      # _, predicted = torch.max(outputs.data, 1)
      # total += labels.size(0)
      # correct += (predicted == labels).sum().item()

      running_loss += loss.item()
  # # mean_val_accuracy = (100 * correct / total)               
  mean_val_loss = ( running_loss )   
  # # print('Validation Accuracy: %d %%' % (mean_val_accuracy)) 
  # print('Validation Loss:'  ,mean_val_loss )
  return( running_loss / len(val_loader))


time: 11 ms (started: 2022-03-22 21:41:12 +00:00)


In [ ]:
def train(dataset, model, hyperparams, device):
  timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
  trainWriter = SummaryWriter('/content/drive/MyDrive/runs/instTrainer/train'.format(timestamp))
  valWriter = SummaryWriter('/content/drive/MyDrive/runs/instTrainer/validation'.format(timestamp))

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=hyperparams.lr)

  # split data
  train_set, val_set = dataset['train'], dataset['val']
  
  train_loader = DataLoader(train_set, batch_size=hyperparams.batchSize, drop_last=True)
  val_loader   = DataLoader(val_set, batch_size=hyperparams.batchSize, drop_last=True)
  # further options: shuffle, num_workers

  for epoch in range(hyperparams.epochs):
    trainLoss = train_epoch(epoch, hyperparams.batchSize, model, criterion, optimizer, train_loader, device, trainWriter)
    valLoss = val_epoch(epoch, hyperparams.batchSize, model, criterion, optimizer, val_loader, device, valWriter)

    trainWriter.add_scalar('loss', trainLoss, epoch)  
    valWriter.add_scalar('loss', valLoss, epoch)  
    
  trainWriter.flush()
  valWriter.flush()
    

time: 11.2 ms (started: 2022-03-22 21:41:12 +00:00)


In [ ]:
def predict(model, dataset, tkn, h, c):
         
  # tensor inputs
  x = np.array([[dataset.tokenizer.word_index[tkn]]])
  inputs = torch.from_numpy(x)
  print('inp')
  print(inputs.shape)
  
  # push to GPU
  inputs = inputs.to(device)

  # get the output of the model
  out, (h, c) = model(inputs, h, c)

  # get the token probabilities
  print('out')
  print(out.shape)
  p = F.softmax(out, dim=1).data
  print('pred')
  print(p.shape)
  print(p.reshape(p.shape[1],).shape)

  p = p.cpu()
  p = p.numpy()
  p = p.reshape(p.shape[1],)

  # get indices of top 3 values
  print(np.argmax(p))
  top_n_idx = p.argsort()[-3:][::-1]

  # randomly select one of the three indices
  sampled_token_index = top_n_idx[random.sample([0,1,2],1)[0]]

  # return the encoded value of the predicted char and the hidden state
  return sampled_token_index, (h, c)


time: 13.8 ms (started: 2022-03-22 21:41:13 +00:00)


In [ ]:
# function to generate text
def sample(model, dataset, size, device, initial):
        
    # push to GPU
    model.to(device)
    
    model.eval()

    # batch size is 1
    h, c = model.init_hidden(1)

    toks = initial
    title = []

    # predict next token
    for t in initial:
      token_idx, (h, c) = predict(model, dataset, t, h, c)
    
    if token_idx > 0:
      token = dataset.tokenizer.index_word[token_idx]
      toks.append(token)
    else:
      token = ';'
    
    title.append(token)

    # predict subsequent tokens
    for i in range(size-1):
        token_idx, (h, c) = predict(model, dataset, toks[-1], h, c)
        if token_idx > 0:
          token = dataset.tokenizer.index_word[token_idx]
          toks.append(token)
        else:
          token = ';'
        title.append(token)

    return ' '.join(title)

time: 12.4 ms (started: 2022-03-22 21:41:13 +00:00)


## Execution

In [ ]:
hyperParams = HyperParams(epochs=50, batchSize=32)
print(hyperParams)

epochs 50
batchSize 32
lr 0.001
ratio train|val|test[0.7, 0.2, 0.1]
hidden_dim 256
num_layers 1
embedding_dim 200

time: 1.36 ms (started: 2022-03-22 21:41:13 +00:00)


In [ ]:
instSet = InstructionSet(hyperParams, baseFrame)
splitSet = random_split(instSet, [int(hyperParams.ratio[0] * len(instSet)), int(hyperParams.ratio[1] * len(instSet)), int(hyperParams.ratio[2] * len(instSet))], generator=torch.Generator().manual_seed(0))
splitSet = {'train': splitSet[0], 'val': splitSet[1], 'test': splitSet[2]}

AttributeError: ignored

time: 1min 2s (started: 2022-03-22 21:41:13 +00:00)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))

In [ ]:
model = Model3(hyperParams, instSet, device)
model.to(device)
print(model)
# summary(model, (16,53))

In [ ]:
train(splitSet, model, hyperParams, device)

Epoch: 1, loss: 3.47934
Epoch: 2, loss: 2.30626
Epoch: 3, loss: 1.85036
Epoch: 4, loss: 1.61080
Epoch: 5, loss: 1.46837
Epoch: 6, loss: 1.37494
Epoch: 7, loss: 1.31024
Epoch: 8, loss: 1.26205
Epoch: 9, loss: 1.22551
Epoch: 10, loss: 1.19719
Epoch: 11, loss: 1.17344
Epoch: 12, loss: 1.15464
Epoch: 13, loss: 1.13828
Epoch: 14, loss: 1.12577
Epoch: 15, loss: 1.11415
Epoch: 16, loss: 1.10420
Epoch: 17, loss: 1.09501
Epoch: 18, loss: 1.08751
Epoch: 19, loss: 1.07986
Epoch: 20, loss: 1.07383
Epoch: 21, loss: 1.06840
Epoch: 22, loss: 1.06360
Epoch: 23, loss: 1.05879
Epoch: 24, loss: 1.05476
Epoch: 25, loss: 1.05162
Epoch: 26, loss: 1.04824
Epoch: 27, loss: 1.04435
Epoch: 28, loss: 1.04166
Epoch: 29, loss: 1.03866
Epoch: 30, loss: 1.03588
Epoch: 31, loss: 1.03428
Epoch: 32, loss: 1.03249
Epoch: 33, loss: 1.03064
Epoch: 34, loss: 1.02836
Epoch: 35, loss: 1.02763
Epoch: 36, loss: 1.02467
Epoch: 37, loss: 1.02484
Epoch: 38, loss: 1.02381
Epoch: 39, loss: 1.02162
Epoch: 40, loss: 1.02058
Epoch: 41

In [ ]:
# sample(model, titleSet, 6, device, initial=['dry', 'penne', 'pasta', 'broccoli', 'sun', 'dried', 'tomatoes', 'packed', 'in', 'oil', 'garlic', 'cloves', 'cheddar', 'cheese', 'salt', 'black', 'pepper'])
seq = splitSet['test'][np.random.randint(0, len(splitSet['test']))][0].tolist()

def remove_values_from_list(the_list, val):
   return [instSet.t300kenizer.index_word[value] for value in the_list if value != val]

seq = remove_values_from_list(seq, 0)
print(seq)

sample(model, instSet, 300, device, initial=seq)

['olive', 'oil', 'onions', 'pepper', 'courgettes', 'aubergines', 'cherry', 'tomatoes', 'pasta', 'sauce', "goat's", 'cheese', 'basil', 'leaves', 'pasta', 'bake', 'with', "goats'", 'cheese']


'melt the butter over a medium heat to medium and simmer over medium heat without stirring once mixture has softened add pepper and shrimp to brown stir for about 5 7 to 4 minutes drain and rinse the lentils with cold water in small bowl stir the mayonnaise lemon zest oregano and salt bring back into the sauce add salt water and to simmer gently stirring until thickened stirring constantly until the sugar melts stir in all purpose orange rind orange sugar salt heat oven 1 1 2 1 4 cup unsweetened chocolate sugar orange juice orange juice lemon juice and sugar beat in eggs one tablespoon of butter over each side using the bottom add your fingers or the top of each pan and add it into a spoon to extract the custard so to add the potatoes to make the bowl and add a small pot over the skillet then add the chicken and cook until it begins to boil pour this mixture over the crust and then place the whole in baking pan place the potatoes and tomatoes on top of the meat with some of it place in

time: 1.96 s (started: 2022-03-25 02:10:59 +00:00)


In [ ]:
pd.DataFrame.from_dict(pd.Series(instSet.tokenizer.word_index))

,0
the,1
and,2
a,3
in,4
to,5
...,...
kaga,62904
thn,62905
gym,62906
uncontaminated,62907


time: 27.6 ms (started: 2022-03-25 02:11:01 +00:00)


# Tensorboard visualization

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=/content/drive/MyDrive/runs/instTrainer

Reusing TensorBoard on port 6006 (pid 1117), started 2:20:56 ago. (Use '!kill 1117' to kill it.)

<IPython.core.display.Javascript object>

time: 9.8 ms (started: 2022-03-25 02:11:01 +00:00)
